In [4]:
from datetime import datetime
import pandas as pd

##helper functions
def str2date(ss):
    date = ss.split('+')[0]
    return  datetime.strptime(date, '%Y-%m-%dT%H:%M:%S')

def date2offset(ss):
    start = str2date('2018-12-02T15:53:11+01:00')
    
    return  (datetime.now() - str2date(ss) ).days



In [1]:
import re, time
import json
import requests


#hash this as we'll be using it a lot
def get_page(url,params={}):
    return session_requests.get(
        url,
        params=params,
        headers = dict(referer=url))

session_requests = requests.session()
login_url = 'https://www.vinted.fr/member/general/login?ref_url='
tt = session_requests.get(login_url).content.decode('utf-8')
pp = re.compile('name="csrf-token" content="(.*?)"')
mm = pp.search(tt)

from vinted_creds import creds
payload = creds
payload["authenticity_token"] =  mm.group(1)

result = session_requests.post( 'https://www.vinted.fr/member/general/login.json',
                               json=json.dumps(payload),
                               headers=dict(referer=login_url)
                              )

print( result.content.decode('utf-8'))

{"status":500,"error":"Internal Server Error"}


In [7]:
def get_items4member(member_id='34003258'):
    base_url = 'https://www.vinted.fr/api/v2/users/'
    
    r = get_page(base_url + '%s/'%member_id)
    user_data = r.json()['user']
    
    num_items = user_data['item_count']
    if num_items == 0: 
        return user_data, []

    url = base_url + '%s/items'%member_id
    stuff = []
    
    for k in range(1, num_items//48 + 2):
        print('page', k)
        r = get_page(url, 
                     params={'page': k, 'per_page': 48})
        stuff.extend( r.json()['items'])
        
    return user_data, stuff



id_='34003258'
metadata, junk = get_items4member(member_id=id_)
    

page 1
page 2
page 3
page 4
page 5


In [8]:
def items2cols(stuff):

    my_keys = ['id', 'view_count', 'created_at_ts', 'price_numeric', 
              'title', 'description','path']

    col_names = ['id', 'views',  'created', 'price', 
               'title', 'description','path', 'off']

    rows = [[ art[x] for x in my_keys ] for art in stuff ]

    cols = list(zip(* rows))
    
    cols.append([date2offset(x) for x in cols[2] ])
    cols = {x : y for x,y in zip(col_names, cols)}
    cols['created'] = [x.split('T')[0] for x in cols['created']]
    cols['price'] =[float(x) for x in cols['price']]
    return cols

cols = items2cols(junk)

today = datetime.strftime(datetime.now(), '%Y-%m-%d')
with open('%s_%s.json'%(id_, today),'w') as fp:
    json.dump(cols, fp)
    
df_art = pd.DataFrame( cols)


In [9]:
df_art.sort_values('off', ascending=True)[:10]


,id,views,created,price,title,description,path,off
0,513820824,36,2020-06-11,10.0,Robe longue femme 🍂,Taille 38,/femmes/maxi/513820824-robe-longue-femme,0
29,512654353,3,2020-06-10,5.0,Jupe femme morgan. 🍂,Taille 42,/femmes/autres-jupes/512654353-jupe-femme-morgan,1
40,512656146,11,2020-06-10,10.0,Petit pull vintage 🍂,Taille 34,/femmes/autres-pull-overs-and-sweat-shirts/512...,1
24,512652031,2,2020-06-10,18.0,Sublime robe vintage 🍂,Taille 36,/femmes/midi/512652031-sublime-robe-vintage,1
23,512379486,8,2020-06-10,10.0,Robe longue femme🍂,Taille 38,/femmes/maxi/512379486-robe-longue-femme,1
22,512650127,7,2020-06-10,13.0,Jeans slim femme 🍂,Taille 40,/femmes/jeans-skinny/512650127-jeans-slim-femme,1
20,512260824,8,2020-06-10,10.0,Blouse vintage 🍂,Taille 40,/femmes/blouses/512260824-blouse-vintage,1
19,512262336,8,2020-06-10,15.0,Haut femme 🍂,Taille 40,/femmes/autres-hauts/512262336-haut-femme,1
18,512263559,4,2020-06-10,10.0,Pull femme 🍂,Taille 34,/femmes/autres-pull-overs-and-sweat-shirts/512...,1
17,512281164,11,2020-06-10,8.0,Blouse femme 🍂,Taille 36,/femmes/blouses/512281164-blouse-femme,1


In [11]:
def get_friends_for(member='34003258'):
    r = get_page('https://www.vinted.fr/member/general/followers/' + member)
    px = re.compile('class="follow__name".*?(\d+).*?>(.*?)<')
    return list( set(px.findall(r.content.decode('utf-8'))) )


get_friends_for()

[('37077921', 'tabarnaak'),
 ('36853843', 'marbi35'),
 ('36823744', 'emedemeri'),
 ('36852530', 'maissas220807'),
 ('36626702', 'leptr_marion76'),
 ('36760346', 'jessicaflore'),
 ('36596025', 'davidtorales'),
 ('36964212', 'amel_dressing'),
 ('36951275', 'merla.shop'),
 ('36784210', 'tropic_lola'),
 ('36943701', 'vide_dressing_1euro'),
 ('36698804', 'aina.fraau'),
 ('36805591', 'candice_tt'),
 ('36643666', 'shadionira'),
 ('36629698', 'nawaal.ss'),
 ('36622952', 'mycloset_rv'),
 ('36767256', 'lovely.style1'),
 ('36864322', 'gcris44'),
 ('36619713', 'glo.m'),
 ('37103733', 'besmallahd'),
 ('36952764', 'mouniamoni'),
 ('36668914', 'oceane.knck'),
 ('36779682', 'juanluisbocanegra'),
 ('36715533', 'anagonzalezz'),
 ('37131662', 'noalena2005'),
 ('36822719', 'alluye'),
 ('36617271', 'mariamolto3'),
 ('37061867', 'luvihe1'),
 ('37214955', 'linessamolenaar'),
 ('36603909', 'lauramadrid1987')]

In [ ]:
<a class="follow__name" data-ch="ufv" href="/member/31290234-bibou0087">bibou0087</a>

In [18]:
_keys = [ 'login', 'gender', 'city',
    'item_count', 'given_item_count', 
    'created_at', 'last_loged_on_ts' ]

def get_user_data(member_id):
    
    r = get_page('https://www.vinted.fr/api/v2/users/%s/'%member_id)
    user_data = r.json()['user']
    return [ user_data[k] for k in _keys ]
    

data = []

for member_id, name  in friends[:3]:
    print(name)
    data.append(get_user_data(member_id))
    time.sleep(3)
    

freezer38
karimus.nebili
bibou0087


In [19]:
cols = dict( zip( _keys, list(zip(*data)) ) )
for f, t in zip((0,1), ['created_at', 'last_loged_on_ts' ]):
    cols[t] = [ x.split('T')[f] for x in cols[t] ]
pd.DataFrame(cols)

,login,gender,city,item_count,given_item_count,created_at,last_loged_on_ts
0,freezer38,M,Saint-Joseph-de-Rivière,5,3,2019-11-28,18:01:36+02:00
1,karimus.nebili,F,Grenoble,94,72,2018-01-20,17:22:39+02:00
2,bibou0087,F,Béziers,764,90,2019-11-13,17:21:02+02:00
